In [1]:
%pwd

'f:\\Medibot-GenAI\\research'

In [2]:
import os
os.chdir("../")


In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# Extract the Data from the PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents 

In [5]:
extracted_data = load_pdf_file(data='Data/')

In [6]:
# extracted_data

In [7]:
# spli the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [9]:
# text_chunks

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
# Dowload the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\Thamindu\AppData\Local\Temp\ipykernel_8360\543302852.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Thamindu\AppData\Local\r-miniconda\envs\medibot_clean\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query_result = embeddings.embed_query("hellow Workd")
print("Length", len(query_result))

Length 384


In [14]:
from dotenv import load_dotenv
load_dotenv()


True

In [15]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
COHERE_API_KEY = os.environ.get('COHERE_API_KEY')

In [16]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medibot"

if index_name not in pc.list_indexes():
    pc.create_index(
            name=index_name,
            dimension=384,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1") # Specify your desired cloud and region
        )

In [17]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [18]:
# Embed each chunk upsert the embeddngs into our pinecone index
from langchain_pinecone import PineconeVectorStore

dosearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)



In [19]:
# Load Existing Index
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into our Pinecode index
dosearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [20]:
dosearch

In [21]:
# This here do is return similarity things. but 3 things
retriever = dosearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [22]:
retrieved_doc = retriever.invoke("what is acne ?")

In [56]:
retrieved_doc

[Document(id='e455e216-0407-42ab-86e6-321d5b3b5877', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='014cb2ac-d24f-4d36-a5b4-1733d33f749f', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM 

In [57]:
import cohere
from langchain_cohere import ChatCohere

# Raw Cohere client (optional if you also need SDK)
co = cohere.ClientV2(COHERE_API_KEY)

# Wrap Cohere as a LangChain LLM
llm = ChatCohere(model="command-r", temperature=0)



In [61]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a medical assistant. "
    "Answer ONLY using the provided context. "
    "If the answer is not in the context, say: "
    "'I'm sorry, I don't have information about that.' "
    "Never make up answers.\n\n{context}"
)


promt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [62]:
quection_answer_chain = create_stuff_documents_chain(llm, promt)
print(llm)
print(promt)

rag_chain = create_retrieval_chain(retriever, quection_answer_chain)

client=<cohere.client.Client object at 0x0000020ECC034EB0> async_client=<cohere.client.AsyncClient object at 0x0000020ECC035B10> model='command-r' temperature=0.0 cohere_api_key=SecretStr('**********')
input_variables=['context', 'input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are a medical assistant. Answer ONLY using the provided context. If the answer is not in the context, say: 'I'm sorry, I don't have information about that.' Never make up answers.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [ ]:
response = rag_chain.invoke({"input" : "What is acne ?"})
print(response["answer"])

I'm sorry, I don't have information about the current US president in the provided context.
